In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from qiskit_metal import MetalGUI, designs
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6

In [3]:
%metal_heading Create a Transmon Qubit in Qiskit Metal

In [4]:
design = designs.DesignPlanar({}, True)
gui = MetalGUI(design)

In [5]:
conn_pads = dict(
    connection_pads = dict(
        readout = dict(loc_W=0, loc_H=-1),
        coupler1 = dict(loc_W=-1, loc_H=1),
        coupler2 = dict(loc_W=1, loc_H=1)
    )
)
q1 = TransmonPocket6(design, "Q1", options=dict(**conn_pads))

gui.rebuild()
gui.autoscale()

In [6]:
%metal_heading Render your design in QElmerRenderer

In [7]:
from qiskit_metal.renderers.renderer_elmer.elmer_renderer import QElmerRenderer

# Instantiate the Elmer renderer
elmer_renderer = QElmerRenderer(design)

# Elmer renderer uses the Gmsh renderer to
# generate a mesh for the design
# Set initial parameters for meshing in Gmsh (IMPORTANT step!!)
elmer_renderer.gmsh.options.mesh.min_size = "3um"
elmer_renderer.gmsh.options.mesh.max_size = "30um"

In [8]:
# Render the design
elmer_renderer.render_design(open_pins=[("Q1", "coupler1"), ("Q1", "coupler2"), ("Q1", "readout")],
                             skip_junctions=True)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 41 (Line)
Info    : [  0%] Meshing curve 44 (Line)
Info    : [  0%] Meshing curve 4 (Line)
Info    : [  0%] Meshing curve 43 (Line)
Info    : [  0%] Meshing curve 42 (Line)
Info    : [  0%] Meshing curve 3 (Line)
Info    : [  0%] Meshing curve 1 (Line)
Info    : [  0%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 45 (Line)
Info    : [ 10%] Meshing curve 46 (Line)
Info    : [ 10%] Meshing curve 48 (Line)
Info    : [ 10%] Meshing curve 105 (Line)
Info    : [ 10%] Meshing curve 107 (Line)
Info    : [ 10%] Meshing curve 106 (Line)
Info    : [ 10%] Meshing curve 108 (Line)
Info    : [ 20%] Meshing curve 109 (Line)
Info    : [ 20%] Meshing curve 110 (Line)
Info    : [ 20%] Meshing curve 111 (Line)
Info    : [ 20%] Meshing curve 112 (Line)
Info    : [ 20%] Meshing curve 113 (Line)
Info    : [ 10%] Meshing curve 47 (Line)
Info    : [ 20%] Meshing curve 114 (Line)
Info    : [ 20%] Meshing curve 115 (Line)
Info    : [ 20%] Meshing c

In [9]:
# View the generated mesh
elmer_renderer.launch_gmsh_gui()

-------------------------------------------------------
Version       : 4.10.3
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 20220526
Build host    : gmsh.info
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blas[petsc] Blossom Cgns DIntegration Dlopen DomHex Eigen[contrib] Fltk Gmm[contrib] Hxt Jpeg Kbipack Lapack[petsc] LinuxJoystick MathEx[contrib] Med Mesh Metis[contrib] Mmg Mpeg Netgen ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom PETSc Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.4.0
PETSc version : 3.14.4 (real arithmtic)
OCC version   : 7.6.1
MED version   : 4.1.0
Packaged by   : geuzaine
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


In [10]:
# Export the generated mesh
elmer_renderer.export_mesh()

Info    : Writing './out.msh'...
Info    : Done writing './out.msh'


In [11]:
# Add a solution setup to solve for the capacitance matrix
elmer_renderer.add_solution_setup('capacitance')

In [12]:
# Run the simulation in ElmerFEM
elmer_renderer.run('capacitance')

11:01AM 48s INFO [run]: Running ElmerGrid on input mesh from Gmsh...
11:01AM 50s INFO [run]: Running ElmerSolver for solver type: 'capacitance'


In [13]:
# Display the capacitnce matrix obtained after the simulation
elmer_renderer.capacitance_matrix

,Q1_readout_connector_pad,Q1_coupler1_connector_pad,Q1_coupler2_connector_pad,Q1_pad_top,Q1_pad_bot,ground_plane
Q1_readout_connector_pad,58.684061,-0.207751,-0.207891,-1.738512,-17.042738,-39.487168
Q1_coupler1_connector_pad,-0.207751,54.208860,-0.281858,-14.852005,-1.579939,-37.287306
Q1_coupler2_connector_pad,-0.207891,-0.281858,54.514793,-14.875394,-1.581328,-37.568322
Q1_pad_top,-1.738512,-14.852005,-14.875394,105.363752,-35.215269,-38.682572
Q1_pad_bot,-17.042738,-1.579939,-1.581328,-35.215269,99.501590,-44.082317
ground_plane,-39.487168,-37.287306,-37.568322,-38.682572,-44.082317,300.000000


In [14]:
elmer_renderer.save_capacitance_matrix("cap_matrix.txt")
elmer_renderer.close()

Info    : Clearing all models and views...
Info    : Done clearing all models and views


In [15]:
%metal_heading Perform LOM 2.0 Analysis

In [16]:
import numpy as np

from qiskit_metal.renderers.renderer_elmer.elmer_renderer import load_capacitance_matrix_from_file
from qiskit_metal.analyses.quantization.lom_core_analysis import CompositeSystem, Cell, Subsystem 

from scipy.constants import speed_of_light as c_light

import matplotlib.pyplot as plt
%matplotlib inline

11:03AM 24s INFO [__init__]: TransmonBuilder with system_type TRANSMON registered to QuantumSystemRegistry
11:03AM 24s INFO [__init__]: FluxoniumBuilder with system_type FLUXONIUM registered to QuantumSystemRegistry
11:03AM 24s INFO [__init__]: TLResonatorBuilder with system_type TL_RESONATOR registered to QuantumSystemRegistry
11:03AM 24s INFO [__init__]: LumpedResonatorBuilder with system_type LUMPED_RESONATOR registered to QuantumSystemRegistry


In [17]:
cap_matrix = load_capacitance_matrix_from_file("cap_matrix.txt")

In [18]:
# Define cells
opt1 = dict(
    node_rename = {'Q1_coupler1_connector_pad': 'coupler1', 'Q1_coupler2_connector_pad': 'coupler2', 'Q1_readout_connector_pad': 'readout'}, 
    cap_mat = cap_matrix,
    ind_dict = {('Q1_pad_top', 'Q1_pad_bot'): 12},  # junction inductance in nH
    jj_dict = {('Q1_pad_top', 'Q1_pad_bot'): 'j1'},
    cj_dict = {('Q1_pad_top', 'Q1_pad_bot'): 2}, # junction capacitance in fF
)
cell_1 = Cell(opt1)

In [19]:
# Define subsystems
# subsystem 1: Transmon
transmon = Subsystem(name='transmon1_sys', sys_type='TRANSMON', nodes=['j1'])

# subsystem 2: Coupler 1
q_opts = dict(
    f_res = 7.4, # resonator dressed frequency in GHz
    Z0 = 50, # characteristic impedance in Ohm
    vp = 0.404314 * c_light # phase velocity 
)
coup1 = Subsystem(name='coup1_sys', sys_type='TL_RESONATOR', nodes=['coupler1'], q_opts=q_opts)

# subsystem 3: Coupler 2
q_opts = dict(
    f_res = 7.2, # resonator dressed frequency in GHz
    Z0 = 50, # characteristic impedance in Ohm
    vp = 0.404314 * c_light # phase velocity 
)
coup2 = Subsystem(name='coup2_sys', sys_type='TL_RESONATOR', nodes=['coupler2'], q_opts=q_opts)

# subsystem 4: Readout
q_opts = dict(
    f_res = 7, # resonator dressed frequency in GHz
    Z0 = 50, # characteristic impedance in Ohm
    vp = 0.404314 * c_light # phase velocity 
)
readout = Subsystem(name='readout_sys', sys_type='TL_RESONATOR', nodes=['readout'], q_opts=q_opts)

In [20]:
composite_sys = CompositeSystem(
    subsystems=[transmon, coup1, coup2, readout], 
    cells=[cell_1], 
    grd_node='ground_plane',
    nodes_force_keep=['coupler1', 'coupler2', 'readout']
)

In [21]:
cg = composite_sys.circuitGraph()
print(cg)

node_jj_basis:
-------------

['j1', 'Q1_pad_bot', 'coupler1', 'coupler2', 'readout']

nodes_keep:
-------------

['j1', 'coupler1', 'coupler2', 'readout']


L_inv_k (reduced inverse inductance matrix):
-------------

                j1  coupler1  coupler2  readout
j1        0.083333       0.0       0.0      0.0
coupler1  0.000000       0.0       0.0      0.0
coupler2  0.000000       0.0       0.0      0.0
readout   0.000000       0.0       0.0      0.0

C_k (reduced capacitance matrix):
-------------

                 j1   coupler1   coupler2    readout
j1        70.760064  -6.277767  -6.288227   8.061600
coupler1  -6.277767  52.200386  -2.293361  -2.503380
coupler2  -6.288227  -2.293361  52.500257  -2.506982
readout    8.061600  -2.503380  -2.506982  56.060221




In [22]:
hilbertspace = composite_sys.create_hilbertspace()
print(hilbertspace)

 /home/abeerv/Documents/coding/qiskit-metal/qiskit_metal/analyses/quantization/lom_core_analysis.py: 170
 /home/abeerv/Documents/coding/qiskit-metal/qiskit_metal/analyses/quantization/lom_core_analysis.py: 170
 /home/abeerv/Documents/coding/qiskit-metal/qiskit_metal/analyses/quantization/lom_core_analysis.py: 170


HilbertSpace:  subsystems
-------------------------

Transmon------------| [Transmon_1]
                    | EJ: 13621.792733898432
                    | EC: 284.05043715960124
                    | ng: 0.001
                    | ncut: 22
                    | truncated_dim: 10
                    |
                    | dim: 45


Oscillator----------| [Oscillator_1]
                    | E_osc: 7400.0
                    | l_osc: None
                    | truncated_dim: 3
                    |
                    | dim: 3


Oscillator----------| [Oscillator_2]
                    | E_osc: 7200.0
                    | l_osc: None
                    | truncated_dim: 3
                    |
                    | dim: 3


Oscillator----------| [Oscillator_3]
                    | E_osc: 7000
                    | l_osc: None
                    | truncated_dim: 3
                    |
                    | dim: 3




In [23]:
hilbertspace = composite_sys.add_interaction()
hilbertspace.hamiltonian()

 /home/abeerv/Documents/coding/qiskit-metal/qiskit_metal/analyses/quantization/lom_core_analysis.py: 170
 /home/abeerv/Documents/coding/qiskit-metal/qiskit_metal/analyses/quantization/lom_core_analysis.py: 170
 /home/abeerv/Documents/coding/qiskit-metal/qiskit_metal/analyses/quantization/lom_core_analysis.py: 170


Quantum object: dims = [[10, 3, 3, 3], [10, 3, 3, 3]], shape = (270, 270), type = oper, isherm = True
Qobj data =
[[-10912.9500518 +0.00000000e+00j      0.        +9.29573831e-02j
       0.        +0.00000000e+00j ...      0.        +0.00000000e+00j
       0.        +0.00000000e+00j      0.        +0.00000000e+00j]
 [     0.        -9.29573831e-02j  -3912.9500518 +0.00000000e+00j
       0.        +1.31461592e-01j ...      0.        +0.00000000e+00j
       0.        +0.00000000e+00j      0.        +0.00000000e+00j]
 [     0.        +0.00000000e+00j      0.        -1.31461592e-01j
    3087.0499482 +0.00000000e+00j ...      0.        +0.00000000e+00j
       0.        +0.00000000e+00j      0.        +0.00000000e+00j]
 ...
 [     0.        +0.00000000e+00j      0.        +0.00000000e+00j
       0.        +0.00000000e+00j ...  58435.87888161+0.00000000e+00j
       0.        +4.49897978e+02j      0.        +0.00000000e+00j]
 [     0.        +0.00000000e+00j      0.        +0.00000000e+00j
   

In [24]:
hamiltonian_results = composite_sys.hamiltonian_results(hilbertspace, evals_count=30)


system frequencies in GHz:
--------------------------
{'transmon1_sys': 5.247600222577874, 'coup1_sys': 7.404753940653035, 'coup2_sys': 7.201632111026363, 'readout_sys': 7.004729558275996}

Chi matrices in MHz
--------------------------
               transmon1_sys  coup1_sys  coup2_sys  readout_sys
transmon1_sys    -322.260345  -1.464554  -1.098688    -2.508974
coup1_sys          -1.464554   1.872002   0.058791    -0.005386
coup2_sys          -1.098688   0.058791   1.810844    -0.000021
readout_sys        -2.508974  -0.005386  -0.000021     2.471589


In [25]:
composite_sys.compute_gs()

                 j1   coupler1   coupler2    readout
j1         0.000000  82.151909  80.099124 -92.957572
coupler1  82.151909   0.000000  15.243893   9.623917
coupler2  80.099124  15.243893   0.000000   9.380654
readout  -92.957572   9.623917   9.380654   0.000000

In [26]:
transmon.h_params

defaultdict(dict,
            {'j1': {'EJ': 13621.792733898432,
              'EC': 284.05043715960124,
              'Q_zpf': 3.204353268e-19,
              'default_charge_op': Operator(op=array([[-22,   0,   0, ...,   0,   0,   0],
                     [  0, -21,   0, ...,   0,   0,   0],
                     [  0,   0, -20, ...,   0,   0,   0],
                     ...,
                     [  0,   0,   0, ...,  20,   0,   0],
                     [  0,   0,   0, ...,   0,  21,   0],
                     [  0,   0,   0, ...,   0,   0,  22]]), add_hc=False)}})

In [28]:
gui.main_window.close()

True